In [1]:
import numpy as np
import pandas as pd

import string
import re
import gc

In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

D:\Projects\OCDBot\thirdParty\winpython\python-3.5.4.amd64\Lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train = pd.read_csv('data/train.csv', encoding='utf-8')
train.dropna(inplace=True)
train = train[train.sentence_id != 492799]

#### Prepare full data

In [4]:
vectorizer = CountVectorizer (
    analyzer='char_wb',
    ngram_range=(1, 1),
    lowercase=False
)

In [5]:
%time data_ngrams = vectorizer.fit_transform(train['before'].values)

Wall time: 3min 6s


In [6]:
data_ngrams.shape

(10574501, 2659)

In [7]:
labels = np.array(train['after'].values != train['before'].values, dtype=np.uint8)

enc = LabelEncoder()
labels = enc.fit_transform(labels)

#### Prepare train data

In [8]:
sample_sizes = {
    "PUNCT" : 25000,
    "PLAIN" : 30000,
    "CARDINAL" : 20000,
    "LETTERS" : 20000,
    "DATE" : 20000,
    "ORDINAL" : 15000,
    "MEASURE" : 15000
}

train.reset_index(inplace=True)
train_data = pd.DataFrame(columns=train.columns)
sample_max_size = 10000

for c in train['class'].unique():
    class_sample = train[train['class'] == c]
    sample_size = sample_sizes[c] if c in sample_sizes else sample_max_size
    if len(class_sample) > sample_max_size:
        train_data = train_data.append(class_sample.sample(sample_size))
    else:
        train_data = train_data.append(class_sample)

In [9]:
train_ngrams = vectorizer.transform(train_data['before'].values)

In [10]:
labels_train = np.array(train_data['after'].values != train_data['before'].values, dtype=np.uint8)
labels_train = enc.transform(labels_train)

### Train XGBoost classifier 

In [11]:
xtr, xcv, ytr, ycv = train_test_split(train_ngrams, labels_train, test_size = 0.05, random_state = 42)
feature_names = ['f_{}'.format(i) for i in range(train_ngrams.shape[1])]

In [12]:
dtrain = xgb.DMatrix(xtr, label=ytr, feature_names=feature_names)
dvalid = xgb.DMatrix(xcv, label=ycv, feature_names=feature_names)

In [28]:
params = [("objective", "binary:logistic"),
          ("booster", "gbtree"),
          ("nthread", 3),
          ("eta", 0.01),
          ("max_depth", 6),
          ("subsample", 0.9),
          ("min_child_weight", 1),
          ("colsample_bytree", 0.7),
          ("eval_metric", 'error'),
          ("eval_metric", 'auc'),
         ]
num_rounds = 5000
stop = 40

In [20]:
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

In [29]:
gbm = xgb.train(params, dtrain, num_rounds, evals=watchlist, early_stopping_rounds=stop)

[0]	train-error:0.14725	train-auc:0.888141	valid-error:0.153066	valid-auc:0.883877
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[1]	train-error:0.120473	train-auc:0.92562	valid-error:0.124546	valid-auc:0.922994
[2]	train-error:0.115964	train-auc:0.937068	valid-error:0.120167	valid-auc:0.933591
[3]	train-error:0.10603	train-auc:0.947352	valid-error:0.111408	valid-auc:0.944138
[4]	train-error:0.114739	train-auc:0.947325	valid-error:0.119526	valid-auc:0.944084
[5]	train-error:0.122553	train-auc:0.948638	valid-error:0.126896	valid-auc:0.945534
[6]	train-error:0.120552	train-auc:0.948544	valid-error:0.123478	valid-auc:0.945524
[7]	train-error:0.110989	train-auc:0.948558	valid-error:0.114612	valid-auc:0.945572
[8]	train-error:0.10743	train-auc:0.948574	valid-error:0.112156	valid-auc:0.945416
[9]	train-error:0.105058	train-auc:0.949266	valid-error:0.110553	valid-auc:0.946311
[10]	train-error:0.10

[96]	train-error:0.091739	train-auc:0.967339	valid-error:0.095172	valid-auc:0.966325
[97]	train-error:0.092076	train-auc:0.967351	valid-error:0.095172	valid-auc:0.966324
[98]	train-error:0.092138	train-auc:0.967365	valid-error:0.095279	valid-auc:0.966329
[99]	train-error:0.091554	train-auc:0.967358	valid-error:0.094638	valid-auc:0.966331
[100]	train-error:0.090941	train-auc:0.967365	valid-error:0.094211	valid-auc:0.966337
[101]	train-error:0.091042	train-auc:0.96736	valid-error:0.094424	valid-auc:0.966336
[102]	train-error:0.090946	train-auc:0.967373	valid-error:0.094317	valid-auc:0.966338
[103]	train-error:0.090941	train-auc:0.967384	valid-error:0.094211	valid-auc:0.966378
[104]	train-error:0.090946	train-auc:0.967381	valid-error:0.094211	valid-auc:0.966376
[105]	train-error:0.090952	train-auc:0.967383	valid-error:0.094211	valid-auc:0.966382
[106]	train-error:0.091042	train-auc:0.967397	valid-error:0.094424	valid-auc:0.966379
[107]	train-error:0.091452	train-auc:0.967397	valid-error:0

[192]	train-error:0.083784	train-auc:0.980304	valid-error:0.086306	valid-auc:0.979643
[193]	train-error:0.08379	train-auc:0.980308	valid-error:0.086306	valid-auc:0.979637
[194]	train-error:0.083807	train-auc:0.980308	valid-error:0.086306	valid-auc:0.979637
[195]	train-error:0.083812	train-auc:0.98031	valid-error:0.086306	valid-auc:0.979635
[196]	train-error:0.083812	train-auc:0.980306	valid-error:0.086306	valid-auc:0.979621
[197]	train-error:0.083812	train-auc:0.980267	valid-error:0.086306	valid-auc:0.979553
[198]	train-error:0.083823	train-auc:0.980272	valid-error:0.086306	valid-auc:0.979557
[199]	train-error:0.083807	train-auc:0.980278	valid-error:0.086306	valid-auc:0.97956
[200]	train-error:0.083818	train-auc:0.980284	valid-error:0.086306	valid-auc:0.979572
[201]	train-error:0.083784	train-auc:0.980338	valid-error:0.086306	valid-auc:0.979651
[202]	train-error:0.083784	train-auc:0.980417	valid-error:0.086306	valid-auc:0.979819
[203]	train-error:0.083773	train-auc:0.981231	valid-error

[288]	train-error:0.071601	train-auc:0.985186	valid-error:0.075198	valid-auc:0.984803
[289]	train-error:0.071601	train-auc:0.985192	valid-error:0.075198	valid-auc:0.984813
[290]	train-error:0.07159	train-auc:0.985191	valid-error:0.075198	valid-auc:0.984809
[291]	train-error:0.071573	train-auc:0.985197	valid-error:0.075198	valid-auc:0.984807
[292]	train-error:0.07159	train-auc:0.986024	valid-error:0.075198	valid-auc:0.985788
[293]	train-error:0.07159	train-auc:0.985999	valid-error:0.075198	valid-auc:0.98576
[294]	train-error:0.071427	train-auc:0.986004	valid-error:0.075091	valid-auc:0.985769
[295]	train-error:0.071427	train-auc:0.986021	valid-error:0.075091	valid-auc:0.985788
[296]	train-error:0.069397	train-auc:0.986026	valid-error:0.073061	valid-auc:0.985792
[297]	train-error:0.06865	train-auc:0.986029	valid-error:0.072314	valid-auc:0.985797
[298]	train-error:0.068526	train-auc:0.986065	valid-error:0.072314	valid-auc:0.985837
[299]	train-error:0.068644	train-auc:0.986071	valid-error:0

[384]	train-error:0.056905	train-auc:0.988045	valid-error:0.060457	valid-auc:0.98827
[385]	train-error:0.05622	train-auc:0.988049	valid-error:0.059603	valid-auc:0.988276
[386]	train-error:0.056231	train-auc:0.988525	valid-error:0.059603	valid-auc:0.988731
[387]	train-error:0.05613	train-auc:0.988525	valid-error:0.059282	valid-auc:0.988717
[388]	train-error:0.05419	train-auc:0.988529	valid-error:0.056825	valid-auc:0.988718
[389]	train-error:0.05419	train-auc:0.988532	valid-error:0.056825	valid-auc:0.988723
[390]	train-error:0.05419	train-auc:0.988958	valid-error:0.056825	valid-auc:0.989225
[391]	train-error:0.05419	train-auc:0.988956	valid-error:0.056825	valid-auc:0.989225
[392]	train-error:0.054179	train-auc:0.988958	valid-error:0.056825	valid-auc:0.989228
[393]	train-error:0.054179	train-auc:0.988963	valid-error:0.056825	valid-auc:0.989235
[394]	train-error:0.054291	train-auc:0.988966	valid-error:0.057146	valid-auc:0.989236
[395]	train-error:0.054201	train-auc:0.988968	valid-error:0.0

...

[3911]	train-error:0.010361	train-auc:0.997738	valid-error:0.010681	valid-auc:0.997196
[3912]	train-error:0.010367	train-auc:0.997737	valid-error:0.010681	valid-auc:0.997195
[3913]	train-error:0.010367	train-auc:0.997738	valid-error:0.010681	valid-auc:0.997198
[3914]	train-error:0.010344	train-auc:0.997739	valid-error:0.010681	valid-auc:0.997198
[3915]	train-error:0.010344	train-auc:0.997739	valid-error:0.010681	valid-auc:0.997199
[3916]	train-error:0.010344	train-auc:0.99774	valid-error:0.010681	valid-auc:0.9972
[3917]	train-error:0.010333	train-auc:0.99774	valid-error:0.010681	valid-auc:0.9972
[3918]	train-error:0.010333	train-auc:0.99774	valid-error:0.010681	valid-auc:0.9972
[3919]	train-error:0.010333	train-auc:0.99774	valid-error:0.010681	valid-auc:0.9972
[3920]	train-error:0.010333	train-auc:0.99774	valid-error:0.010681	valid-auc:0.9972
[3921]	train-error:0.010333	train-auc:0.99774	valid-error:0.010681	valid-auc:0.997201
[3922]	train-error:0.010333	train-auc:0.99774	valid-error:0

[4006]	train-error:0.010311	train-auc:0.99777	valid-error:0.010681	valid-auc:0.997216
[4007]	train-error:0.010311	train-auc:0.99777	valid-error:0.010681	valid-auc:0.997216
[4008]	train-error:0.010311	train-auc:0.99777	valid-error:0.010681	valid-auc:0.997216
[4009]	train-error:0.010311	train-auc:0.997771	valid-error:0.010681	valid-auc:0.997216
[4010]	train-error:0.010311	train-auc:0.997771	valid-error:0.010681	valid-auc:0.997215
[4011]	train-error:0.010311	train-auc:0.997771	valid-error:0.010681	valid-auc:0.997216
[4012]	train-error:0.010311	train-auc:0.997771	valid-error:0.010681	valid-auc:0.997216
[4013]	train-error:0.010311	train-auc:0.997771	valid-error:0.010681	valid-auc:0.997216
[4014]	train-error:0.010311	train-auc:0.997774	valid-error:0.010681	valid-auc:0.997217
[4015]	train-error:0.010311	train-auc:0.997774	valid-error:0.010681	valid-auc:0.997217
[4016]	train-error:0.010311	train-auc:0.997774	valid-error:0.010681	valid-auc:0.99722
[4017]	train-error:0.010311	train-auc:0.997774	

[4101]	train-error:0.00972	train-auc:0.997799	valid-error:0.010147	valid-auc:0.997234
[4102]	train-error:0.00972	train-auc:0.997799	valid-error:0.010147	valid-auc:0.997234
[4103]	train-error:0.00972	train-auc:0.997799	valid-error:0.010147	valid-auc:0.997234
[4104]	train-error:0.00972	train-auc:0.997799	valid-error:0.010147	valid-auc:0.997234
[4105]	train-error:0.00972	train-auc:0.997799	valid-error:0.010147	valid-auc:0.997232
[4106]	train-error:0.00972	train-auc:0.9978	valid-error:0.010147	valid-auc:0.99723
[4107]	train-error:0.00972	train-auc:0.9978	valid-error:0.010147	valid-auc:0.99723
[4108]	train-error:0.00972	train-auc:0.9978	valid-error:0.010147	valid-auc:0.997231
[4109]	train-error:0.00972	train-auc:0.997801	valid-error:0.010147	valid-auc:0.997232
[4110]	train-error:0.00972	train-auc:0.9978	valid-error:0.010147	valid-auc:0.997232
[4111]	train-error:0.00972	train-auc:0.997802	valid-error:0.010147	valid-auc:0.997232
[4112]	train-error:0.00972	train-auc:0.997802	valid-error:0.0101

[4196]	train-error:0.009242	train-auc:0.997829	valid-error:0.009827	valid-auc:0.997263
[4197]	train-error:0.009242	train-auc:0.99783	valid-error:0.009827	valid-auc:0.997262
[4198]	train-error:0.009242	train-auc:0.99783	valid-error:0.009827	valid-auc:0.997263
[4199]	train-error:0.009242	train-auc:0.99783	valid-error:0.009827	valid-auc:0.997264
[4200]	train-error:0.009242	train-auc:0.99783	valid-error:0.009827	valid-auc:0.997265
[4201]	train-error:0.009242	train-auc:0.99783	valid-error:0.009827	valid-auc:0.997262
[4202]	train-error:0.009242	train-auc:0.99783	valid-error:0.009827	valid-auc:0.997263
[4203]	train-error:0.009242	train-auc:0.99783	valid-error:0.009827	valid-auc:0.997262
[4204]	train-error:0.009242	train-auc:0.99783	valid-error:0.009827	valid-auc:0.997262
[4205]	train-error:0.009242	train-auc:0.99783	valid-error:0.009827	valid-auc:0.997262
[4206]	train-error:0.009242	train-auc:0.99783	valid-error:0.009827	valid-auc:0.997264
[4207]	train-error:0.009242	train-auc:0.997831	valid-

[4291]	train-error:0.009068	train-auc:0.997862	valid-error:0.0094	valid-auc:0.997278
[4292]	train-error:0.009068	train-auc:0.997863	valid-error:0.0094	valid-auc:0.997278
[4293]	train-error:0.009068	train-auc:0.997863	valid-error:0.0094	valid-auc:0.997278
[4294]	train-error:0.009068	train-auc:0.997864	valid-error:0.0094	valid-auc:0.997286
[4295]	train-error:0.009068	train-auc:0.997864	valid-error:0.0094	valid-auc:0.997286
[4296]	train-error:0.009068	train-auc:0.997864	valid-error:0.0094	valid-auc:0.997283
[4297]	train-error:0.009068	train-auc:0.997864	valid-error:0.0094	valid-auc:0.997283
[4298]	train-error:0.009068	train-auc:0.997864	valid-error:0.0094	valid-auc:0.997283
[4299]	train-error:0.009068	train-auc:0.997864	valid-error:0.0094	valid-auc:0.997285
[4300]	train-error:0.009068	train-auc:0.997864	valid-error:0.0094	valid-auc:0.997285
[4301]	train-error:0.009068	train-auc:0.997864	valid-error:0.0094	valid-auc:0.997283
[4302]	train-error:0.009068	train-auc:0.997865	valid-error:0.0094

### Validate classification accuracy

In [22]:
dtest = xgb.DMatrix(data_ngrams, label=labels, feature_names=feature_names)

In [30]:
%time predictions = gbm.predict(dtest)

Wall time: 30min 4s


In [31]:
predictions

array([ 0.03599444,  0.00104367,  0.03005324, ...,  0.03163639,
        0.00250797,  0.09092531], dtype=float32)

In [32]:
from sklearn.metrics import accuracy_score

for i in range(10):
    predictions_current = predictions > i / 10.0
    print ("0.{}: {}".format(i, accuracy_score(labels, predictions_current)))

0.0: 0.12508240341553706
0.1: 0.9505958720889052
0.2: 0.9646726592583423
0.3: 0.9797295399565427
0.4: 0.990523051631467
0.5: 0.9917951683961257
0.6: 0.9912946246825264
0.7: 0.9910463860185932
0.8: 0.9911289431056841
0.9: 0.9889257185752784


In [41]:
import sys

p = 0.5
for n in range(1, 10000):
    left = n * p - (1 - p)
    right = n * p + p
    sys.stdout.write("\r [{}, {}]".format(left / n, right / n))
    sys.stdout.flush()

 [0.49994999499949994, 0.5000500050005]4]]